## Importing the Dependencies

In [9]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split

## Data extraction and preprocessing

### PIMA diabetes dataset is being used

In [10]:
diabetes_df = pd.read_csv('diabetes.csv')
diabetes_df - diabetes_df.sample(n = len(diabetes_df)) # shuffling the data
diabetes_df.head() # How the data looks

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
# Checking for N/A values
diabetes_df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [12]:
diabetes_df.describe() # statistical measures of the data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [13]:
# Lets count and group the dataframe rows by the targets to get a sense of feature influence and data distributions
print("Outcomes distributions 1-diabetic, 0-non-diabetic: \n",diabetes_df['Outcome'].value_counts())

print('-'*50)

diabetes_df.groupby('Outcome').mean()

Outcomes distributions 1-diabetic, 0-non-diabetic: 
 Outcome
0    500
1    268
Name: count, dtype: int64
--------------------------------------------------


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [14]:
# Preparing values and target
X = diabetes_df.drop('Outcome', axis=1)
y = diabetes_df['Outcome']

X.shape, y.shape

((768, 8), (768,))

In [15]:
# Normalization to prevent unwanted influence of a particular feature
X_norm=(X-X.mean())/X.std() # this remains a pd dataframe
X_norm.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-7.864080e-17,-1.156482e-17,1.272131e-17,3.816392e-17,-4.972874e-17,2.659909e-16,2.451743e-16,2.035409e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.141108e+00,-3.781190e+00,-3.570271e+00,-1.287373e+00,-6.924393e-01,-4.057829e+00,-1.188778e+00,-1.040871e+00
25%,-8.443348e-01,-6.847901e-01,-3.670975e-01,-1.287373e+00,-6.924393e-01,-5.951906e-01,-6.885198e-01,-7.857741e-01
50%,-2.507887e-01,-1.218083e-01,1.495433e-01,1.544326e-01,-4.277835e-01,9.413653e-04,-2.999328e-01,-3.606124e-01
75%,6.395305e-01,6.053764e-01,5.628560e-01,7.186174e-01,4.117396e-01,5.843897e-01,4.659233e-01,6.597757e-01
max,3.904034e+00,2.442886e+00,2.732747e+00,4.918660e+00,6.648507e+00,4.452906e+00,5.879733e+00,4.061069e+00


## Preparing Train and Test splits

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.2, stratify=y, random_state=2)

print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


In [29]:
# Transforming dataset to tensors suitable for pytorch
X_train_tensor = torch.from_numpy(X_train.to_numpy()).float()
X_test_tensor = torch.from_numpy(X_test.to_numpy()).float()
Y_train_tensor = torch.from_numpy(Y_train.to_numpy()).reshape(-1, 1).float()
Y_test_tensor = torch.from_numpy(Y_test.to_numpy()).reshape(-1, 1).float()

X_train_tensor.shape, Y_train_tensor.shape, X_test_tensor.shape, Y_test_tensor.shape

(torch.Size([614, 8]),
 torch.Size([614, 1]),
 torch.Size([154, 8]),
 torch.Size([154, 1]))

## Model preparation and training

In [32]:
class DiabetesClassifier(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.hidden1 = nn.Linear(8, 4)
        self.act1 = nn.ReLU()
        self.output = nn.Linear(4, 1)
        self.act_output = nn.Sigmoid()
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act_output(self.output(x))
        return x

model = DiabetesClassifier()
print(model)

DiabetesClassifier(
  (hidden1): Linear(in_features=8, out_features=4, bias=True)
  (act1): ReLU()
  (output): Linear(in_features=4, out_features=1, bias=True)
  (act_output): Sigmoid()
)


## Loss and Optimizer

In [33]:
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training

In [34]:
n_epochs = 100
batch_size = 10
 
for epoch in range(n_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        Xbatch = X_train_tensor[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = Y_train_tensor[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 1.7258367538452148
Finished epoch 1, latest loss 1.2726655006408691
Finished epoch 2, latest loss 0.7512059211730957
Finished epoch 3, latest loss 0.5935401320457458
Finished epoch 4, latest loss 0.5311040282249451
Finished epoch 5, latest loss 0.5224899649620056
Finished epoch 6, latest loss 0.5137693285942078
Finished epoch 7, latest loss 0.5060229897499084
Finished epoch 8, latest loss 0.4910108745098114
Finished epoch 9, latest loss 0.4901670813560486
Finished epoch 10, latest loss 0.4917377829551697
Finished epoch 11, latest loss 0.4922412037849426
Finished epoch 12, latest loss 0.4925665557384491
Finished epoch 13, latest loss 0.49437588453292847
Finished epoch 14, latest loss 0.49244004487991333
Finished epoch 15, latest loss 0.493877649307251
Finished epoch 16, latest loss 0.4957502484321594
Finished epoch 17, latest loss 0.49515262246131897
Finished epoch 18, latest loss 0.4965129792690277
Finished epoch 19, latest loss 0.4975947141647339
Finished

## Model Evaluation based on accuracy

In [35]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X_train_tensor)
accuracy = (y_pred.round() == Y_train_tensor).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.7442996501922607


In [36]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X_test_tensor)
accuracy = (y_pred.round() == Y_test_tensor).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.7272727489471436


In [40]:
input_data = (5,166,72,19,175,25.8,0.587,51)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = (input_data_reshaped - input_data_reshaped.mean())/input_data_reshaped.std()
print(std_data)

prediction = model(torch.tensor(std_data).float())
print(prediction)

if (prediction[0] <= .5):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[[-0.91034909  1.5613241   0.1182354  -0.69542099  1.69949217 -0.59102734
  -0.9780975  -0.20415675]]
tensor([[0.1993]], grad_fn=<SigmoidBackward0>)
The person is not diabetic
